<a href="https://colab.research.google.com/github/criarNFT/Dream/blob/main/Dream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
!nvidia-smi

In [ ]:
#@title
%pip install diffusers
%pip install transformers scipy ftfy accelerate

In [ ]:
#@title
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained( model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = lambda images, clip_input: (images, False)
print(pipe.device)

In [ ]:
#@title  { vertical-output: true }
prompt = "an astronaut riding a horse on space." #@param {type:"string"}
negativeprompt = "" #@param {type:"string"}
width = 512 #@param {type:"slider", min:512, max:768, step:8}
height=512 #@param {type:"slider", min:512, max:768, step:8}
guidance_scale=8 #@param {type:"slider", min:1, max:20, step:1}
num_inference_steps = 50 #@param {type:"slider", min:15, max:300, step:5}
seed=1024 #@param {type:"integer"}

generator = torch.Generator("cuda").manual_seed(seed)
image = pipe(prompt=prompt, negative_prompt=negativeprompt, generator=generator, width=width, height=height, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images[0]

image

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

num_cols = 3 #@param {type:"integer"}
num_rows = 3 #@param {type:"integer"}

prompt_grid = [prompt] * num_cols
negativepromptgrid = [negativeprompt] * num_cols
generator = torch.Generator("cuda").manual_seed(seed)

all_images = []
for i in range(num_rows):
  images = pipe(prompt=prompt_grid, negative_prompt=negativepromptgrid, generator=generator, width=width, height=height, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid